<a href="https://colab.research.google.com/github/DeokhuiHan/Data-Analysis-with-Open-Source/blob/main/%EC%98%A4%ED%94%88%EC%86%8C%EC%8A%A4_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EB%B6%84%EC%84%9D_4%EA%B0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



# 오픈소스 기반 데이터 분석 4강 - 데이터 수집


## 4-1 CSV 파일 읽기

In [21]:
import requests
import pandas as pd

api_key = "554a6d55756a75723438656b43596f"
base_url = f"http://openapi.seoul.go.kr:8088/{api_key}/json/energyUseDataSummaryInfo/1/5/"

data_list = []

for year in range(2015, 2025):
    for month in range(1, 13):
        url = f"{base_url}/{year}/{month}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            data_list.append(data)
        else:
            print("Error")

print("호출성공 & 데이터 수집 완료!")

df = pd.DataFrame(data_list)
print(df.info())
print(df.head())

import time

df['CRNT_YM'] = df['BASE_YEAR'] + df['BASE_MONTH']
df['DATE'] = pd.to_datetime(df['CRNT_YM'], format='%Y%m')

df['year'] = df['DATE'].dt.year
def get_season(month):
    if 3 <= month <= 5:
        return "봄"
    elif 6 <= month <= 8:
        return "여름"
    elif 9 <= month <= 11:
        return "가을"
    else:
        return "겨울"

df['month'] = df['DATE'].dt.month
df['season'] = df['month'].apply(get_season)
print(df[['year', 'season']])



호출성공 & 데이터 수집 완료!
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 1 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   energyUseDataSummaryInfo  120 non-null    object
dtypes: object(1)
memory usage: 1.1+ KB
None
                            energyUseDataSummaryInfo
0  {'list_total_count': 1241, 'RESULT': {'CODE': ...
1  {'list_total_count': 1241, 'RESULT': {'CODE': ...
2  {'list_total_count': 1241, 'RESULT': {'CODE': ...
3  {'list_total_count': 1241, 'RESULT': {'CODE': ...
4  {'list_total_count': 1241, 'RESULT': {'CODE': ...


KeyError: 'BASE_YEAR'

In [ ]:
import requests
import pandas as pd

service_key = "554a6d55756a75723438656b43596f"
start_year = 2015
end_year = 2024

def fetch_energy_data(api_key, start_year, end_year):
    base_url = "http://openapi.seoul.go.kr:8088/554a6d55756a75723438656b43596f/json/energyUseDataSummaryInfo/1/5/{year}/{month}"
    all_data = []
    dates_to_fetch = []
    for year in range(start_year, end_year + 1):
        for month in range(1, 13):
            dates_to_fetch.append((year, month))

    for year, month in tqdm(dates_to_fetch, desc="데이터 수집 중"):
        month_str = str(month).zfill(2)
        url = base_url.format(key=api_key, year=year, month=month_str)

        try:
            response = requests.get(url)
            response.raise_for_status()
            json_data = response.json()

            if 'energyUseDataSummaryInfo' in json_data and 'row' in json_data['energyUseDataSummaryInfo']:
                row_data = json_data['energyUseDataSummaryInfo']['row'][0]
                row_data['BASE_YEAR'] = str(year)
                row_data['BASE_MONTH'] = month_str
                all_data.append(row_data)
            else:
                print(f"[{year}-{month_str}] 데이터 구조 오류 또는 데이터 없음")

        except requests.exceptions.RequestException as e:
            print(f"[{year}-{month_str}] API 호출 오류: {e}")

        except requests.exceptions.JSONDecodeError as e:
            print(f"[{year}-{month_str}] JSON 디코딩 오류: {e}")

    return all_data

collected_data = fetch_energy_data(SERVICE_KEY, START_YEAR, END_YEAR)

print("\n--- 수집 완료된 데이터 (일부) ---")
print(collected_data[:5])
print(f"총 {len(collected_data)}개월 분 데이터 수집 완료.")

df = pd.DataFrame(collected_data)
print(df.info())

# 데이터의 타입 변환 (필수 전처리)
# 에너지 사용량 컬럼은 분석을 위해 숫자형(float)으로 변환
energy_cols = ['CRNT_ELEC_QTY', 'CRNT_GAS_QTY', 'CRNT_WTR_QTY', 'CRNT_HEAT_QTY']

# 결측치를 0으로 채우고 (선택적) 데이터 타입을 float으로 변환
df[energy_cols] = df[energy_cols].fillna(0).astype(float)

# 'CRNT_YM' (연월) 컬럼을 날짜/시간 형식으로 변환하여 전처리 기반 마련
df['CRNT_YM'] = df['BASE_YEAR'] + df['BASE_MONTH']
df['DATE'] = pd.to_datetime(df['CRNT_YM'], format='%Y%m')

# 1. 연도(year) 컬럼 추가
df['year'] = df['DATE'].dt.year

# 2. 계절(season) 컬럼 추가
def get_season(month):
    if 3 <= month <= 5:
        return "봄"
    elif 6 <= month <= 8:
        return "여름"
    elif 9 <= month <= 11:
        return "가을"
    else:
        return "겨울"

df['month'] = df['DATE'].dt.month
df['season'] = df['month'].apply(get_season)


# 변환 결과 확인

print(df[['year', 'season']])

import matplotlib.pyplot as plt
import seaborn as sns

# 시각화 설정 (한글 폰트 설정)
# Matplotlib에서 한글이 깨지는 것을 방지하기 위해 폰트 설정이 필요합니다.
##plt.rcParams['font.family'] = 'Malgun Gothic' # Windows 사용자
# plt.rcParams['font.family'] = 'AppleGothic' # macOS 사용자
##plt.rcParams['axes.unicode_minus'] = False # 마이너스 폰트 깨짐 방지
# Matplotlib의 기본 폰트 설정이 없는 환경이라면, 위의 설정을 실행해야 합니다.

# 학번 뒤 4자리 (TODO: 본인의 학번으로 변경)
LAST_4_DIGITS = "3765"


### 3-1. 연도별 에너지 사용 총 사용량 변화


# 에너지 사용 총량 컬럼 생성
df['TOTAL_ENERGY_QTY'] = df['CRNT_ELEC_QTY'] + df['CRNT_GAS_QTY'] + df['CRNT_WTR_QTY'] + df['CRNT_HEAT_QTY']

# 연도별 총 사용량 집계
yearly_total = df.groupby('year')['TOTAL_ENERGY_QTY'].sum().reset_index()

# 선 그래프 시각화
plt.figure(figsize=(10, 6))
sns.lineplot(x='year', y='TOTAL_ENERGY_QTY', data=yearly_total, marker='o')

# 그래프 제목 및 레이블 설정
title = f"연도별 에너지 사용 총 사용량 변화 - {LAST_4_DIGITS}"
plt.title(title, fontsize=16)
plt.xlabel("연도", fontsize=12)
plt.ylabel("총 에너지 사용량 (합산)", fontsize=12)
plt.grid(True, linestyle='--', alpha=0.7)
plt.xticks(yearly_total['year']) # x축 틱을 연도별로 표시
plt.tight_layout()

# 그래프 저장 (파일 이름도 학번에 맞춰 변경)
file_name_3_1 = f"yearly_total_energy_trend_{3765}.png"
plt.savefig(file_name_3_1)
plt.show()

print(f"3-1. 그래프가 {file_name_3_1}로 저장되었습니다.")


##**시각화 코드와 생성된 그래프 캡처:**



### 3-2. 계절별 가스 사용량 평균


# 계절별 가스 사용량 평균 집계
season_avg_gas = df.groupby('season')['CRNT_GAS_QTY'].mean().reindex(['봄', '여름', '가을', '겨울']).reset_index()

# 막대 그래프 시각화
plt.figure(figsize=(8, 6))
bars = sns.barplot(x='season', y='CRNT_GAS_QTY', data=season_avg_gas, palette='coolwarm')

# 각 막대에 수치 표시
for bar in bars.patches:
    # 막대의 높이 (평균값) 가져오기
    height = bar.get_height()
    plt.annotate(f'{height:,.0f}', # 천 단위 구분 기호 표시
                 (bar.get_x() + bar.get_width() / 2, height),
                 ha='center', va='bottom',
                 xytext=(0, 5),
                 textcoords='offset points',
                 fontsize=10)

# 그래프 제목 및 레이블 설정
plt.title("계절별 가스 사용량 평균", fontsize=16)
plt.xlabel("계절", fontsize=12)
plt.ylabel("평균 가스 사용량", fontsize=12)
plt.tight_layout()

# 그래프 저장
file_name_3_2 = f"season_avg_gas_consumption_{LAST_4_DIGITS}.png"
plt.savefig(file_name_3_2)
plt.show()

print(f"3-2. 그래프가 {file_name_3_2}로 저장되었습니다.")



데이터 수집 중: 100%|██████████| 120/120 [00:42<00:00,  2.81it/s]


--- 수집 완료된 데이터 (일부) ---
[{'YEAR': '2015', 'MON': '01', 'MM_TYPE': '개인', 'CNT': '767791', 'EUS': '193784708', 'EUS1': '194781915', 'EUS2': '204969429', 'ECO2_1': '-6090964', 'ECO2_2': '-2582568.736', 'GUS': '59133720', 'GUS1': '57163993', 'GUS2': '68297619', 'GCO2_1': '-3597086', 'GCO2_2': '-8057472.64', 'WUS': '12819757.886', 'WUS1': '12723680.426', 'WUS2': '12899476.73', 'WCO2_1': '8179.308', 'WCO2_2': '2715.530256', 'HUS': '22740838.937', 'HUS1': '23400055.303', 'HUS2': '27090493.875', 'HCO2_1': '-2504435.652', 'HCO2_2': '-33660084.213069', 'REG_DATE': '2015-06-04 17:03:55.0', 'BASE_YEAR': '2015', 'BASE_MONTH': '01'}, {'YEAR': '2015', 'MON': '02', 'MM_TYPE': '개인', 'CNT': '774620', 'EUS': '189974230', 'EUS1': '193611430', 'EUS2': '200055533', 'ECO2_1': '-6859251.5', 'ECO2_2': '-2908322.636', 'GUS': '56487358', 'GUS1': '59353536', 'GUS2': '66191173', 'GCO2_1': '-6284996.5', 'GCO2_2': '-14078392.16', 'WUS': '12656888.218', 'WUS1': '12713146.172', 'WUS2': '12948410.081', 'WCO2_1': '-173

KeyError: "None of [Index(['CRNT_ELEC_QTY', 'CRNT_GAS_QTY', 'CRNT_WTR_QTY', 'CRNT_HEAT_QTY'], dtype='object')] are in the [columns]"

In [ ]:
import pandas as pd

## data.csv 파일 읽기
df = pd.read_csv('data.csv', encoding='utf-8', sep=',', header=0,
                 index_col=None, skiprows=None, nrows=None)
print(df)

           날짜    체중  골격근량  체지방량
0  2025.02.06  64.7  30.0  11.1
1  2025.02.04  64.0  29.3  11.6


## 4-2 JSON 파일 읽기



In [ ]:
import json
import pandas as pd

## data.json 파일 출력
with open('data.json', mode='r',encoding='utf-8') as f:
    data = json.load(f)
    print(data)
## data.json 파일 DataFrame 읽기
df= pd.read_json('data.json', orient='records', encoding='utf-8', )
print(df)

{'매출데이터': [{'월': '2025-01', '매출액': 1000000, '비용': 700000, '이익': 300000}, {'월': '2025-02', '매출액': 1200000, '비용': 800000, '이익': 400000}, {'월': '2025-03', '매출액': 1500000, '비용': 900000, '이익': 600000}]}
                                               매출데이터
0  {'월': '2025-01', '매출액': 1000000, '비용': 700000,...
1  {'월': '2025-02', '매출액': 1200000, '비용': 800000,...
2  {'월': '2025-03', '매출액': 1500000, '비용': 900000,...


## 4-3 텍스트 파일 읽기 및 데이터 추출

In [ ]:
import re

## 파일(callcenter20250301.log) 오픈 및 읽기
with open('callcenter20250301.log', 'r', encoding='utf-8') as f:
    content = f.read()
## 주민등록번호 패턴 생성
masked_content = pattern = re.compile(r'(\d{6})-(\d{7})')

## 주민등록번호 마스킹
masked_content = pattern.sub(r'\1-*******', content)

## 마스킹된 파일(callcenter20250301_masked.log) 오픈 및 쓰기
with open('callcenter20250301_masked.log', mode='w') as f:
    f.write(masked_content)

print("주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.")

주민등록번호 마스킹 완료. 'callcenter20250301_masked.log.txt' 파일로 저장되었습니다.


## 4-4 Open-Meteo의 무료 날씨 API를 통한 특정 지역 온도 조회

In [ ]:
import requests
import json
url = "http://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m"
params = {
    "latitude": "37.58638333",
    "longitude": "127.0203333",
    "current": "temperature_2m"
}


try:
    ## URL 및 파라미터 전송
    response = requests.get(url, params=params)
    response.raise_for_status()

    ## JSON 데이터 읽기
    data = response.json()

    print("API 응답:", data)
    print("서울시 종로구의 현재 온도는 : {0}{1}입니다")

except requests.exceptions.RequestException as e:
    print(f"API호출실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

API 응답: {'latitude': 37.6, 'longitude': 127.0, 'generationtime_ms': 0.056624412536621094, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 29.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-21T05:00', 'interval': 900, 'temperature_2m': 14.4}}
서울시 종로구의 현재 온도는 : {0}{1}입니다


In [ ]:
import requests
import json

url = "https://api.open-meteo.com/v1/forecast?=&=&current=temperature_2m"
params = {
    "latitude": "36.5040736",
    "longitude": "127.2494855",
    "current": "temperature_2m"
}

try:
    ## URL 및 파라미터 전송
    response = requests.get(url, params=params)
    response.raise_for_status()

    ## JSON 데이터 읽기
    data = response.json()

    print("API 응답:", data)
    print("세종시의 현재 온도는 : {0}{1} 입니다.".format(data['current']['temperature_2m'], data['current_units']['temperature_2m']))

except requests.exceptions.RequestException as e:
    print(f"API 호출 실패: {e}")
except json.JSONDecodeError as e:
    print(f"JSON 파싱 실패: {e}")

API 응답: {'latitude': 36.5, 'longitude': 127.25, 'generationtime_ms': 0.03230571746826172, 'utc_offset_seconds': 0, 'timezone': 'GMT', 'timezone_abbreviation': 'GMT', 'elevation': 32.0, 'current_units': {'time': 'iso8601', 'interval': 'seconds', 'temperature_2m': '°C'}, 'current': {'time': '2025-10-10T02:15', 'interval': 900, 'temperature_2m': 19.7}}
세종시의 현재 온도는 : 19.7°C 입니다.


## 4-5 Selenium과 lxml을 이용한 웹 스크래핑

In [ ]:
!curl -o google-chrome-stable_current_amd64.deb https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!apt install ./google-chrome-stable_current_amd64.deb -y
!pip install selenium webdriver_manager

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  114M  100  114M    0     0  75.7M      0  0:00:01  0:00:01 --:--:-- 75.7M
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
Note, selecting 'google-chrome-stable' instead of './google-chrome-stable_current_amd64.deb'
The following additional packages will be installed:
  libvulkan1 mesa-vulkan-drivers
The following NEW packages will be installed:
  google-chrome-stable libvulkan1 mesa-vulkan-drivers
0 upgraded, 3 newly installed, 0 to remove and 38 not upgraded.
Need to get 10.9 MB/131 MB of archives.
After this operation, 448 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libvulkan1 amd64 1.3.204.1-2 [128 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 mesa-vulkan-drivers amd64 23.2.1-1ubuntu3.1~22.04.3 [10.7

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from lxml import html
import time

chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless')               # 브라우저 창 없이 실행
chrome_options.add_argument('--no-sandbox')             # 보안모드 비활성화 (Colab 필수)
chrome_options.add_argument('--disable-dev-shm-usage')  # 메모리 부족 방지 (Colab 필수)
chrome_options.add_argument('--window-size=1920x1080')  # 창 크기 설정(가상)
chrome_options.add_argument('--disable-gpu')            # GPU 가속 비활성화 (일부 환경 안정성)
chrome_options.binary_location = "/usr/bin/google-chrome-stable"  # Colab용 크롬 경로 지정

## 드라이버 실행
driver = webdriver.Chrome(options=chrome_options)

## 사이트 접속
## url = 'https://professor.knou.ac.kr/jaehwachung/index.do'
url = 'https://github.com/jaehwachung/Data-Analysis-with-Open-Source'
driver.get(url)

## 사이트 접속 대기
time.sleep(2)

## 페이지 제목 출력
page_source = driver.page_source
tree = html.fromstring(page_source)

title_text= tree.xpath('//title/text()')
print(title_text)
## 드라이버 종료
driver.quit()

['GitHub - jaehwachung/Data-Analysis-with-Open-Source: Data Analysis with Open Source']



# 실습 시나리오

## 공공데이터 포털 가입 및 데이터 신청

- [https://www.data.go.kr](https://www.data.go.kr)
- 한국환경공단 에어코리아 대기오염정보 데이터 신청

In [ ]:
import requests

## 데이터 수집 url 및 api key 설정
url = 'http://openapi.seoul.go.kr:8088/554a6d55756a75723438656b43596f/json/energyUseDataSummaryInfo/1/5/{2016}/{1}'
api_key = '554a6d55756a75723438656b43596f'

params = {
    'serviceKey': api_key,
    'returnType': 'json',
    'numOfRows': '100',
    'pageNo': '1',
    'sidoName': '서울',
    'ver': '1.0'
}

## 데이터 수집
response = requests.get(url, params=params)

## 호출 성공/실패 출력
print(response.json())


{'RESULT': {'CODE': 'INFO-200', 'MESSAGE': '해당하는 데이터가 없습니다.'}}
